In [1]:
import os
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

warnings.filterwarnings('ignore')

/home/fm-pc-lt-219/anaconda3/envs/generative_nlp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [3]:
path_to_data = os.path.join(os.path.dirname(os.getcwd()), 'receipe_data', 'indian_food_receipe.csv')

In [5]:
df = pd.read_csv(path_to_data)
print(df.shape)
df.head()

(6169, 12)


,TranslatedRecipeName,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,TranslatedInstructions,receipe_instructions,receipe_instructions_embeddings
0,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",15,30,45,6,Indian,Side Dish,Diabetic Friendly,"To begin making the Masala Karela Recipe,de-se...",How can we make Masala Karela Recipe? To begin...,[-1.30960166e-01 2.34784689e-02 -1.28126666e-...
1,Spicy Tomato Rice (Recipe),"2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",5,10,15,3,South Indian Recipes,Main Course,Vegetarian,"To make tomato puliogere, first cut the tomato...",How can we make Spicy Tomato Rice (Recipe)? To...,[-9.50396582e-02 -9.98032615e-02 -4.78960872e-...
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",20,30,50,4,South Indian Recipes,South Indian Breakfast,High Protein Vegetarian,"To begin making the Ragi Vermicelli Recipe, fi...",How can we make Ragi Semiya Upma Recipe - Ragi...,[-1.19294345e-01 4.82010953e-02 7.38642062e-...
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"500 grams Chicken,2 Onion - chopped,1 Tomato -...",15,30,45,4,Andhra,Lunch,Non Vegeterian,To begin making Gongura Chicken Curry Recipe f...,How can we make Gongura Chicken Curry Recipe -...,[-1.03536166e-01 -6.44470677e-02 -5.50146885e-...
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"1 tablespoon chana dal, 1 tablespoon white ura...",10,20,30,4,Andhra,South Indian Breakfast,Vegetarian,"To make Andhra Style Alam Pachadi, first heat ...",How can we make Andhra Style Alam Pachadi Reci...,[-6.44649193e-02 -3.83207388e-02 -7.17725307e-...


In [6]:
df['TranslatedRecipeName'].value_counts()

TranslatedRecipeName
Masala Karela Recipe                                                       1
Umm Ali Recipe - Egyptian Dessert Recipe With Puff Pastry                  1
Godhumai Idli Recipe (Instant Whole Wheat Flour Idli)                      1
Vazhakkai Poriyal Recipe - Raw banana Roast With Pepper                    1
Medu Vada Recipe (Traditional Fried Urad Dal Vada Using Vada Maker)        1
                                                                          ..
 Soya Chunks Sabzi (Recipe In Hindi)                                       1
Punjabi Mixed Vegetable Kadai Recipe                                       1
Spaghetti Pasta Recipe in Basil Pesto Sauce                                1
Sichuan Dan-Dan Noodles Recipe (Sesame Noodles With Crispy Tofu Recipe)    1
Kashmiri Style Kokur Yakhni Recipe-Chicken Cooked In Yogurt                1
Name: count, Length: 6169, dtype: int64

In [7]:
search_query = """
How can we make a tandoori chicken?
"""

In [8]:
query_vector = model.encode(search_query)

In [9]:
len(query_vector)

384

In [10]:
print(len(query_vector))

384


In [11]:
search_params = {
    "metric_type": "L2",
    "offset": 0, 
    "ignore_growing": False,
    "params": {"nprobe": 5}
    
}

In [12]:
## Import a PyMilvus package

from pymilvus import (
connections,
utility,
Collection,
)

In [13]:
connections.connect(alias = "default", hostname = "localhost", port = 19530)

In [14]:
receipe_collection = Collection('indian_receipe_collections')

In [15]:
receipe_collection.load()

In [16]:
results = receipe_collection.search(
    data = [query_vector],
    anns_field = "receipe_instructions_embeddings",
    param = search_params,
    limit=3,
    expr = None,
    output_fields = ['TranslatedInstructions'],
    consistency_level = "Strong"
    
)

In [17]:
results[0].ids

[819, 1527, 6156]

In [18]:
results[0].distances

[0.6178910732269287, 0.6190319061279297, 0.6323031783103943]

In [20]:
top3_context = []
for result in results[0]:
    print(result.distance)
    context = result.entity.get('TranslatedInstructions')
    context.replace("\\xa0", " ")
    context.replace("\xa0", " ")
    top3_context.append(context)

0.6178910732269287
0.6190319061279297
0.6323031783103943


In [21]:
top3_receipes = "\n\n".join(top3_context)

In [22]:
top3_receipes

"To begin making the Tandoori Chicken Momos Recipe, first let’s make the dough.For the Momo doughIn a mixing bowl, add maida and add water slowly and knead to form a medium stiff dough. The dough should be non sticky, smooth and firm.\xa0Keep the momo dough covered for 2 hours before making filling.To make the Chicken Momo FillingIn a mixing bowl. combine the chicken cubes along with\xa0fresh cream,\xa0tandoori masala,\xa0\xa0garam masala, cumin powder,\xa0red chilli powder, chaat masala, salt and lemon juice and mix well. Set aside for 15 minutes.\xa0Heat a grill pan on medium-high flame, place the marinated chicken pieces and cook for about 3-4 minutes on each side. Ensuring that the chicken momo filling is cooked.\xa0Turn off the heat, transfer the chicken momo filling into a bowl and set aside.\xa0To make the Tandoori Chicken Momos RecipeTake a bamboo dim sum maker, or the regular idli steamer and fill it with water and pre heat it over a medium heat.Divide the momo dough into equa